# Sequence alignment in BioPython

In this labs, we will introduce BioPython facilities to build, parse, and store alignments, both pairwise and multiple. 

## Sequence alignment I/O

Before looking into how to compute an alignment, let's have a look into BioPython routines for manipulation of existing alignments. This comes handy when processing (multiple) sequence alignments comming from resources such as [Pfam](http://pfam-legacy.xfam.org/).

### Read alignment

To manipulate sequence alignment, BioPython features the [Bio.AlignIO](https://biopython.org/docs/1.75/api/Bio.AlignIO.html) package with methods to read a single alignment (`Bio.AlignIO.read()`) or multiple alignments (`Bio.AlignIO.parse()`). `read` returns a single [MultipleSeqAlignment](https://biopython.org/docs/1.75/api/Bio.Align.html#Bio.Align.MultipleSeqAlignment) object while `parse` returns an iterator over `MultipleSeqAlignment` objects. Both methods accept a file hande (or path) and [format](https://biopython.org/docs/1.75/api/Bio.AlignIO.html#file-formats) of the alignment. For example the Pfam alignments are stored in the [Stockholm file format](https://en.wikipedia.org/wiki/Stockholm_format). Let's read in the Coronavirus spike glycoprotein S1 family MSA [PF01600](https://pfam.xfam.org/family/PF01600).

In [1]:
from Bio import AlignIO
alignment = AlignIO.read("{}/PF01600_full.sto".format('data'), "stockholm")
print(alignment)

Alignment with 32 rows and 557 columns
----------------------YQVL-P---DSGEFSDNLFTVG...PK- S5YNL4_9ALPC/219-623
-------------MRANIRNSQ--------------TDVCTTIQ...PQ- H9BR08_9NIDO/50-472
----------------------YSIC-K---NCSGFPDHVFAAG...PK- A0A0U1WHD9_9ALPC/218-633
---------------------V-QNC-T--GNCEEYANNIFSTE...P-T D9J204_9ALPC/237-647
---------------------C-SNCTD---QCASYVANVFVTQ...PS- Q0PL12_9ALPC/21-443
--------------------YP---C-P---TSSPFVSGDCVIK...RQ- V5TFD8_9GAMC/303-694
----------------------YTLC-D---NCTGFPQHVFATM...P-L A0A0U1WHD7_9ALPC/215-642
----------------------YSVC-N---DCAGFPKYVFAVN...P-A B1PHI8_9ALPC/221-631
---------------YEFCEDY-----E---YCTATATNVFAPT...PS- H9TEX4_9ALPC/251-675
----------------------YSVC-D---DCDGFPKYVFAVT...P-P B1PHJ5_9ALPC/221-631
----------------------YTLC-S---NCSGFPQHVFAVG...PK- A0A0U1WHB6_9ALPC/218-633
---------------------V-SNCTD---QCASYVANVFTTQ...PS- SPIKE_CVPPU/248-668
----------------------YSVC-T---ECDGFPKHVFPVL...T-E K4JZP8_9ALPC/219-610
----------------

We can iterate over the records in the alignment, obtaining the individual sequences as `SeqRecord`s (see the first lab for details on `SeqRecord`).

In [2]:
for record in alignment:
    print(record.letter_annotations)
    print(f"{record.id}\n{record.dbxrefs}\n{record.seq}")   


{}
S5YNL4_9ALPC/219-623
[]
----------------------YQVL-P---DSGEFSDNLFTVGDDGSIPP-SFGFNNWFVLSNSSSIISGTVVSNQPLRLT---C--LWPIP-----SSTGALATI---YFNGTN----GA-QCN------------GFDS--NAPFDAIRFNL--NGTLSGHNFVS----GFVLHAANGATLGFSCTNSTDAPYLR-------QIPFGI-GDT-PYYCYLNV---------TTDINSTMSFVGALPLNLREIVIA-SNGDVYMNGYRYFAAGDLSSVDVELPSQQV--FGSTFWTIAFTVFETVLLEVDGTSINRMLYCD--NPL-NRVKCSHTQFDLVDGFYPLT--DVDLAVKPFTF-VTLPTFADHSFVYFNFSLMF-----DDLN----------EDFRLQSFNLTINGQL---------SYCVQSRQFTT-SGSVRTNT------------------NHQFGFYTQRAAS---------NGCPFTIDTLNNYLTFGRICFSFG-ESGAGCGVDVMVESQYNMFKVT---T---IFVSYSEGDIIAGMPK-
{}
H9BR08_9NIDO/50-472
[]
-------------MRANIRNSQ--------------TDVCTTIQQGGFIPS-TFTFPQWYVLTNGSTFLQGEYTLSQPLLAN---A--HFCPR-----KNSDGYWRY---SFNNSCL-FPDH-RCQDHWYDSQNPICLGWNNT-FGLSDNIRINI--NISHDEYQSHG---GYVSLTLESGSVVNITCTNNSDPSTVTL---ATSLLPWARAIDQ-PMYCFANL---------TTGTASQLDFMGMLPPLVSELAFD-RTGGIYINGYRYYLTSALRDVDFKLKRND----TAEYFAVTWANYTDVHLSVDAGAIEKIKYCN--TPL-DRLACDMNVFNLSDGVYSYT--SLEKASVPETF-VTLPVYSNHTYVTINTSYTV-----GS

We can also obtain column-level annotations, i.e. annotations which hold for all the sequences in the alignment. 

In [ ]:
for key in sorted(alignment.column_annotations.keys()):
    print("{}: {}\n".format(key, alignment.column_annotations[key]))

### Write alignment
In order to serialize a `MultipleSeqAlignment` object, we need to call `Bio.AlignIO.write()` method and pass in the alignment object, path to the file and format.

In [ ]:
AlignIO.write(alignment, 'PF01600_serialized.faa', 'fasta')

We can also serialize the alignment directly with the print function.

In [ ]:
print(format(alignment, "clustal"))

### Manipulate alignment

The `MultipleSeqAlignment` has several convenience method to both build and manipulate an existing alignment.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
a = SeqRecord(Seq("AAAACGT"), id="Alpha")
b = SeqRecord(Seq("AAA-CGT"), id="Beta")
c = SeqRecord(Seq("AAAAGGT"), id="Gamma")
align = MultipleSeqAlignment([a, b, c],
                             annotations={"tool": "demo"},
                             column_annotations={"stats": "CCCXCCC"})

In [ ]:
print(list(align))
print(len(align))
print(align.annotations)
print(align.column_annotations)

To add a sequence to an existing alignemnt, we can use the `append` and `extend` mehtods (the sequence lenght must match the MSA length).

In [ ]:
align.append(SeqRecord(Seq("--AAG-T"), id="Delta"))

In [ ]:
list(align)

Alignment can be also extended column-wise:

In [ ]:
list(align + align)

It is possible to slice the alignment both column and row-wise.

In [ ]:
print(align[-1], '\n')
print(align[1:3], '\n')
print(align[1:3, 1:4], '\n')
print(align[:,1:4], '\n')

Note that MSA slicing takes into the account the annotations.

In [ ]:
sliced = align[:, 1:4]
print(sliced)
print(sliced.column_annotations)

Note that the combination of colun-wise extension and slicing enables removal of columns.

In [ ]:
list(align[:, 1:2] + align[:, 3:4])

If a more advanced array manipulation is required, MSA can be converted to a NumPy array.

In [ ]:
import numpy as np
align_array = np.array([list(rec) for rec in align])
print(align_array.shape)
print(align_array[1:2])

### ---- Begin Exercise ----
- Write a function that takes in a Pfam family, loads from the disk the corresponding MSA and computes sum-of-pairs score for each column. To do that, the user will have also to specify the scoring matrix (see below how to work with scoring matrices). The method will compute SP for every column and together with the number of gaps for that columns. The results will be stored in `column_annotations` of the MSA. The MSA with the enriched `column_annotations` will be returned to the user.
- Modify the previous function so that it takes on the input only Pfam ID, and using `requests` it first downloads the Pfam family (check in the *view* option on any Pfam family page in Alignment->Format an alignment and click *Generate* to see how the URL is constructed) and only then computes the SP scores and gap counts as in previous step.
### ---- End Exercise ----

## Obtaining pairwise alignment

BioPython includes two sequence aligners - the "old" [Bio.pairwise2 module](https://biopython.org/docs/latest/api/Bio.pairwise2.html) and the new [PairwiseAligner class](https://biopython.org/docs/latest/api/Bio.Align.html#Bio.Align.PairwiseAlignment) that is part of the [Bio.Align module](https://biopython.org/docs/latest/api/Bio.Align.html). It is suggested to use the `PairwiseAligner` class as it provides faster and more efficient implementation. That said, both aligners should return the same results.

In [ ]:
from Bio import Align
aligner = Align.PairwiseAligner()

In [ ]:
seq1 = "GAACT"
seq2 = "GAT"

aligner = Align.PairwiseAligner(match_score=1.0)
aligner.match_score = 2.0

print(aligner.score(seq1, seq2))

In [ ]:
alignments = aligner.align(seq1, seq2)
for a in alignments:
    print(a)

In [ ]:
aligner.mode

In [ ]:
seq1 = "AGAACTC"
seq2 = "GAACT"
aligner.gap_score = 0
aligner.match_score = 1
for mode in ['global', 'local']:
    aligner.mode = mode
    print(aligner.algorithm)
    alignments = aligner.align(seq1, seq2)
    for a in alignments:
        print(a)

The aligner has a trully fine-grained control over the gap penalties.

In [ ]:
print(aligner)

Opening scores|Extending scores
---|---
query_left_open_gap_score|query_left_extend_gap_score
query_internal_open_gap_score|query_internal_extend_gap_score
query_right_open_gap_score|query_right_extend_gap_score
target_left_open_gap_score|target_left_extend_gap_score
target_internal_open_gap_score|target_internal_extend_gap_score
target_right_open_gap_score|target_right_extend_gap_score

target|	query|	score|
---|---|---|
A|	-|	query left open gap score|
C|	-|	query left extend gap score|
C|	-|	query left extend gap score|
G|	G|	match score|
G|	T|	mismatch score|
G|	-|	query internal open gap score|
A|	-|	query internal extend gap score|
A|	-|	query internal extend gap score|
T|	T|	match score|
A|	A|	match score|
G|	-|	query internal open gap score|
C|	C|	match score|
-|	C|	target internal open gap score|
-|	C|	target internal extend gap score|
C|	C|	match score|
T|	G|	mismatch score|
C|	C|	match score|
-|	C|	target internal open gap score|
A|	A|	match score|
-|	T|	target right open gap score|
-|	A|	target right extend gap score|
-|	A|	target right extend gap score|


Meta-attribute |	Attributes it maps to
---|---
gap_score	|target_gap_score, query_gap_score
open_gap_score|	target_open_gap_score, query_open_gap_score
extend_gap_score|	target_extend_gap_score, query_extend_gap_score
internal_gap_score|	target_internal_gap_score, query_internal_gap_score
internal_open_gap_score|	target_internal_open_gap_score, query_internal_open_gap_score
internal_extend_gap_score|	target_internal_extend_gap_score, query_internal_extend_gap_score
end_gap_score|	target_end_gap_score, query_end_gap_score
end_open_gap_score|	target_end_open_gap_score, query_end_open_gap_score
end_extend_gap_score|	target_end_extend_gap_score, query_end_extend_gap_score
left_gap_score|	target_left_gap_score, query_left_gap_score
right_gap_score| target_right_gap_score, query_right_gap_score
left_open_gap_score|	target_left_open_gap_score, query_left_open_gap_score
left_extend_gap_score|	target_left_extend_gap_score, query_left_extend_gap_score
right_open_gap_score|	target_right_open_gap_score, query_right_open_gap_score
right_extend_gap_score|	target_right_extend_gap_score, query_right_extend_gap_score
target_open_gap_score|	target_internal_open_gap_score, target_left_open_gap_score,
_|target_right_open_gap_score
target_extend_gap_score|	target_internal_extend_gap_score, target_left_extend_gap_score,
_|target_right_extend_gap_score
target_gap_score|	target_open_gap_score, target_extend_gap_score
query_open_gap_score|	query_internal_open_gap_score, query_left_open_gap_score,
_|query_right_open_gap_score
query_extend_gap_score|	query_internal_extend_gap_score, query_left_extend_gap_score,
_|query_right_extend_gap_score
query_gap_score|	query_open_gap_score, query_extend_gap_score
target_internal_gap_score|	target_internal_open_gap_score, target_internal_extend_gap_score
target_end_gap_score|	target_end_open_gap_score, target_end_extend_gap_score
target_end_open_gap_score|	target_left_open_gap_score, target_right_open_gap_score
target_end_extend_gap_score|	target_left_extend_gap_score, target_right_extend_gap_score
target_left_gap_score|	target_left_open_gap_score, target_left_extend_gap_score
target_right_gap_score|	target_right_open_gap_score, target_right_extend_gap_score
query_end_gap_score|	query_end_open_gap_score, query_end_extend_gap_score
query_end_open_gap_score|	query_left_open_gap_score, query_right_open_gap_score
query_end_extend_gap_score|	query_left_extend_gap_score, query_right_extend_gap_score
query_internal_gap_score|	query_internal_open_gap_score, query_internal_extend_gap_score
query_left_gap_score|	query_left_open_gap_score, query_left_extend_gap_score
query_right_gap_score|	query_right_open_gap_score, query_right_extend_gap_score

It is even possible to have a general gap scoring function

In [ ]:
def my_gap_score_function(start, length):
    if start==2:
        return -1000
    else:
        return -1 * length

for query_gap_score in [0, my_gap_score_function]:
    print(query_gap_score)
    aligner.query_gap_score = query_gap_score
    alignments = aligner.align("AACCCTT", "AATT")
    for alignment in alignments:
        print(alignment)

For protein sequences, it is reasonable to use a substitution matrix. BioPython is distributed with plenty of substitution matrices (including PAM and BLOSSUM) which are available via the [scoring_matrices](https://biopython.org/docs/latest/api/Bio.Align.substitution_matrices.html) subpackage of `Bio.Align`. To find out which matrices are available, we can call the `load` method without an argument (the matrices are stored as flat files in `Bio/Align/scoring_matrices/data`). The same method is then used to load a specific matrix.

In [ ]:
from Bio.Align import substitution_matrices
substitution_matrices.load()

In [ ]:
m = substitution_matrices.load("BLOSUM62")
print(m)
print("A->R substitution score: {}".format(m['A', 'R']))

Let's use the BLOSSUM62 matrix to find an alignment between spike glycoprotein in SARS ([P59594](https://www.uniprot.org/uniprot/P59594)) and spike glycoprotein in bat corona virus ([R9QTA0](https://www.uniprot.org/uniprot/R9QTA0)). But first, let's see what happens if we do not specify the scoring matrix.

In [ ]:
from Bio import SeqIO
s_sars = SeqIO.read('data/spike_sars_cv.faa', 'fasta')
s_bat = SeqIO.read('data/spike_bat_cv.faa', 'fasta')

In [ ]:
aligner = Align.PairwiseAligner()
alignments = aligner.align(s_sars.seq, s_bat.seq)

In [ ]:
print(len(alignments))

We got so many alignments because the scoring system results in plenty of alignments with the same score. Let's inspect.

In [ ]:
print(aligner)

In [ ]:
print(alignments[0])
print(alignments[1])

Copy the above output into a text editor to get rid of the strange text wrapping (might look OK depending on your Notebook's environment). Alternatively, we can convert the output into an MSA object and format it as a clustal alignment (we are using string splitting here because `Bio.Align.PairwiseAlignment` [slicing](https://biopython.org/docs/latest/api/Bio.Align.html#Bio.Align.PairwiseAlignment.__getitem__) does not seem to be implemented at the time of writing this notebook).

In [ ]:
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
aln_str = str(alignments[0])
a = SeqRecord(Seq(aln_str.splitlines()[0]), id="sars")
b = SeqRecord(Seq(aln_str.splitlines()[1]), id="bat")
msa = MultipleSeqAlignment([a, b])
print(format(msa, "clustal"))

Now let's use the BLOSSUM62 matrix.

In [ ]:
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")


In [ ]:
aligner.open_gap_score = -11
aligner.end_open_gap_score = -11
aligner.extend_gap_score = -1
aligner.end_extend_gap_score = -1

In [ ]:
alignments = aligner.align(s_sars.seq, s_bat.seq)

In [ ]:
len(alignments)

### ---- Begin Exercise ----

- Iterate over the alignments and print out the alignments togeter with percentage identity (you can use the `substitutions` property which is an `np` 2D array)
- Compare the results with what you get from UniProt's BLAST similarity search for the `SARS` protein and with what you get from using [EMBOSS Needle](https://www.ebi.ac.uk/Tools/psa/emboss_needle/). Are they any different?

### ---- End Exercise ----

## Obtaining MSA

As there is no single agreed-upon standard for how to align multiple sequences there exists no algorithm implemented directly in BioPython. Instead, BioPython supports running external tools (which need to be installed on the target system) and wrapping their outputs into an MSA alignment which can then be further processed in BioPython. The wrappers are defined in the [Bio.Align.Applications](https://biopython.org/docs/latest/api/Bio.Align.Applications.html) module.

In [ ]:
import Bio.Align.Applications
dir(Bio.Align.Applications)

In [ ]:
from Bio.Align.Applications import ClustalwCommandline
clustalw_cline = ClustalwCommandline(r"c:\Program Files (x86)\ClustalW2\clustalw2.exe", infile="data/PF01600_full_length_sequences.fasta")

In [ ]:
stdout, stderr = clustalw_cline()

In [ ]:
print(stdout)

The alignment is, in case of ClustalW, actually written into an output file so we can then read it as we would do with any MSA.

In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("data/PF01600_full_length_sequences.aln", "clustal")
print(alignment)

The tree based on which the MSA is created is also available and can be visualized.

In [ ]:
from Bio import Phylo
tree = Phylo.read("data/PF01600_full_length_sequences.dnd", "newick")
Phylo.draw_ascii(tree)